In [1]:
import numpy as np

In [ ]:
n = 10000
m = 100

eta = 5005


In [57]:
def L_uni_mean(t, Y):
    mu1, mu2 = np.mean(Y[:t]), np.mean(Y[t:])
    return np.sum((Y[:t] - mu1)**2) + np.sum((Y[t:] - mu2)**2)

def fixed_grid(Y, m):
    n = len(Y)
    res = np.zeros(m + 1)
    res[0], res[-1] = np.infty, np.infty
    for i in range(1, m):
        res[i] = L_uni_mean(i * (n // m), Y)
    return res

In [66]:
def divide_conquer(jump = [2,3], verbose = False):
    T = 2
    eps = 5
    n = np.array([50000 + eps, 50000 - eps])
    p = 1

    delta = np.random.uniform(jump[0], jump[1], T)
    delta[0] = 0
    beta = np.stack([np.random.uniform(0,1,p) + delta[i] for i in range(T)])

    diff = np.zeros(T - 1)
    for t in range(1, T):
        diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5

    y_train = np.concatenate([np.random.normal(beta[i], 1, n[i]) for i in range(T)])
    y_train_joint = y_train.reshape((-1, 1)).squeeze()

    # divide and conquer
    m = 100
    alpha = sum(n) // m
    true_ix = np.zeros(T - 1)
    for t in range(T - 1):
        true_ix[t] = sum(n[:(t + 1)]) / alpha
    
    L_seq = fixed_grid(y_train_joint, m)

    E = np.argsort(L_seq)[:2]
    
    eta_l, eta_r = min(E), max(E)

    if verbose:
        print(diff)
        print(true_ix)
        print(E)
        
    if eta_l <= true_ix <= eta_r:
        return 1
    else:
        return 0

In [64]:
divide_conquer(verbose = True)

[2.29509481]
[50.005]
[50 51]


1

In [69]:
np.random.seed(0)
B = 100
record = np.zeros(B)
for b in range(B):
    record[b] = divide_conquer(verbose = False)
print(sum(record) / B)

0.57


In [74]:
np.random.seed(0)
B = 100
record = np.zeros(B)
for b in range(B):
    record[b] = divide_conquer(jump = [20,30], verbose = False)
print(sum(record) / B)

1.0
